<a href="https://colab.research.google.com/github/jenny12138/CNN_SinglePatient/blob/main/MEG_IED_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Set Up and Load All Data from Drive


In [ ]:
## Import the necessary packages/modules/libraries

import copy
import scipy.io
import numpy as np
from matplotlib import pyplot as plt 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import metrics
from tensorflow import keras
from sklearn.utils import shuffle
import collections
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
import pickle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## Mount Google Drive to this notebook so that we can import the processed EEG data

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Import positive data (epileptiform epochs) from MATLAB files

# Import from run 3
EST_run03_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/EST_03.mat')
EST_run03_F = EST_run03_matlab['EST_03F']

# Import from run 4
EST_run04_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/EST_04.mat')
EST_run04_F = EST_run04_matlab['EST_04F']

# Import from run 5
EST_run05_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/EST_05.mat')
EST_run05_F = EST_run05_matlab['EST_05F']

# Import from run 6
EST_run06_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/EST_06.mat')
EST_run06_F = EST_run06_matlab['EST_06F']

In [ ]:
## Import negative data (non-epileptiform epochs) from MATLAB files

# Import from run 3
negative_run03_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/NEG_03.mat')
negative_run03_F = negative_run03_matlab['NEG_03F']

# Import from run 4
negative_run04_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/NEG_04.mat')
negative_run04_F = negative_run04_matlab['NEG_04F']

# Import from run 5
negative_run05_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/NEG_05.mat')
negative_run05_F = negative_run05_matlab['NEG_05F']

# Import from run 6
negative_run06_matlab = scipy.io.loadmat('/content/drive/MyDrive/Pt0090_Data_MEGCleaned/NEG_06.mat')
negative_run06_F = negative_run06_matlab['NEG_06F']

In [ ]:
## Make copies of the loaded data

# Run 3 
EST_run03_raw_data = copy.deepcopy(EST_run03_F)
negative_run03_raw_data = copy.deepcopy(negative_run03_F)
print(type(EST_run03_raw_data))
print(type(negative_run03_raw_data))

# Run 4
EST_run04_raw_data = copy.deepcopy(EST_run04_F)
negative_run04_raw_data = copy.deepcopy(negative_run04_F)
print(type(EST_run04_raw_data))
print(type(negative_run04_raw_data))

# Run 5 
EST_run05_raw_data = copy.deepcopy(EST_run05_F)
negative_run05_raw_data = copy.deepcopy(negative_run05_F)
print(type(EST_run05_raw_data))
print(type(negative_run05_raw_data))

# Run 6
EST_run06_raw_data = copy.deepcopy(EST_run06_F)
negative_run06_raw_data = copy.deepcopy(negative_run06_F)
print(type(EST_run06_raw_data))
print(type(negative_run06_raw_data))

## Part 2: Processing the Data for Neural Network

In [ ]:
## We need to transpose raw_data. Currently, the shape of raw_data is [nChannels, nSamples]. In order to use the epoching function 
## (taken from Raymundo Cassani at https://github.com/MuSAELab/amplitude-modulation-analysis-module/blob/master/am_analysis/am_analysis.py),
## we need to transpose raw_data such that its shape becomes [nSamples, nChannels]

# Run 3
EST_run03_raw_data_transposed = np.transpose(EST_run03_raw_data)
print(EST_run03_raw_data_transposed.shape) #(154368, 394)
negative_run03_raw_data_transposed = np.transpose(negative_run03_raw_data)
print(negative_run03_raw_data_transposed.shape) #(238336, 394)

# Run 4
EST_run04_raw_data_transposed = np.transpose(EST_run04_raw_data)
print(EST_run04_raw_data_transposed.shape) #(60416, 394)
negative_run04_raw_data_transposed = np.transpose(negative_run04_raw_data)
print(negative_run04_raw_data_transposed.shape) #(57344, 394)

# Run 5
EST_run05_raw_data_transposed = np.transpose(EST_run05_raw_data)
print(EST_run05_raw_data_transposed.shape) #(61952, 394)
negative_run05_raw_data_transposed = np.transpose(negative_run05_raw_data)
print(negative_run05_raw_data_transposed.shape) #(39936, 394)

# Run 6
EST_run06_raw_data_transposed = np.transpose(EST_run06_raw_data)
print(EST_run06_raw_data_transposed.shape) #(156416, 394)
negative_run06_raw_data_transposed = np.transpose(negative_run06_raw_data)
print(negative_run06_raw_data_transposed.shape) #(122368, 394)

In [ ]:
## By Raymundo Cassani, used with permission from: https://github.com/MuSAELab/amplitude-modulation-analysis-module/blob/master/am_analysis/am_analysis.py

def epoching(data, samples_epoch, samples_overlap = 0):
    """Divide an array in a colletion of smaller arrays
    
    Divides the `data` provided as [n_samples, n_channels] using the 
    `size_epoch` indicated (in samples) and the `overlap_epoch` between 
    consecutive epochs.
   
    Parameters
    ----------
    data : 2D array 
        with shape (n_samples, n_channels)
    samples_epochs : 
        number of samples in smaller epochs
        
    samples_overlap : 
        number of samples for ovelap between epochs (Default 0)
    Returns
    -------
    epochs : 3D array 
        with shape (samples_epoch, n_channels, n_epochs)
    
    remainder : 2D array 
        with the remaining data after last complete epoch
    
    ix_center : 1D array
        indicates the index tha corresponds to the center of the nth epoch.
    """ 
    # input 'data' as 2D matrix [samples, columns]
    try:
        data.shape[1]
    except IndexError:
        data = data[:, np.newaxis]
    
    # number of samples and number of channels
    n_samples, n_channels = data.shape

    # Size of half epoch
    half_epoch = np.ceil(samples_epoch / 2 )

    # Epoch shift   
    samples_shift = samples_epoch - samples_overlap

    # Number of epochs
    n_epochs =  int(np.floor( (n_samples - samples_epoch) / float(samples_shift) ) + 1 )
    if n_epochs == 0:
        return np.array([]), data, np.array([])

    #markers indicates where the epoch starts, and the epoch contains samples_epoch rows
    markers = np.asarray(range(0,n_epochs)) * samples_shift
    markers = markers.astype(int)

    #Divide data in epochs
    epochs = np.zeros((samples_epoch, n_channels, n_epochs))
    ix_center = np.zeros((n_epochs,1))

    for i_epoch in range(0,n_epochs):
        epochs[:,:,i_epoch] = data[ markers[i_epoch] : markers[i_epoch] + samples_epoch ,:]
        ix_center[i_epoch] = markers[i_epoch] -1 + half_epoch
        
    if ( (markers[-1] + samples_epoch) < n_samples): 
        remainder = data[markers[-1] + samples_epoch : n_samples, :]
    else:
        remainder = np.asarray([])
    
    return epochs, remainder, ix_center.astype(int)

In [ ]:
## Now, we need to epoch the positive examples. We will use the epoching function by Raymundo (from 
## https://github.com/MuSAELab/amplitude-modulation-analysis-module/blob/master/am_analysis/am_analysis.py), taken with permission

## epochs_EST_run0# is a numpy ndarray with shape [number of samples, number of channels, number of 1-second epochs]
## Note that the number of samples should be 256 since the data in Brainstorm has been downsampled to 256 Hz

# Run 3
EST_run03_epoched = epoching(EST_run03_raw_data_transposed, 256, 0)
epochs_EST_run03 = EST_run03_epoched[0]
print(epochs_EST_run03.shape) #(256, 394, 603) 
negative_run03_epoched = epoching(negative_run03_raw_data_transposed, 256, 0)
epochs_negative_run03 = negative_run03_epoched[0]
print(epochs_negative_run03.shape) #(256, 394, 931) 

# Run 4
EST_run04_epoched = epoching(EST_run04_raw_data_transposed, 256, 0)
epochs_EST_run04 = EST_run04_epoched[0]
print(epochs_EST_run04.shape) #(256, 394, 236)
negative_run04_epoched = epoching(negative_run04_raw_data_transposed, 256, 0)
epochs_negative_run04 = negative_run04_epoched[0]
print(epochs_negative_run04.shape) #(256, 394, 224) 

# Run 5
EST_run05_epoched = epoching(EST_run05_raw_data_transposed, 256, 0)
epochs_EST_run05 = EST_run05_epoched[0]
print(epochs_EST_run05.shape) #(256, 394, 242) 
negative_run05_epoched = epoching(negative_run05_raw_data_transposed, 256, 0)
epochs_negative_run05 = negative_run05_epoched[0]
print(epochs_negative_run05.shape) #(256, 394, 156) 

# Run 6
EST_run06_epoched = epoching(EST_run06_raw_data_transposed, 256, 0)
epochs_EST_run06 = EST_run06_epoched[0]
print(epochs_EST_run06.shape) #(256, 394, 611) 
negative_run06_epoched = epoching(negative_run06_raw_data_transposed, 256, 0)
epochs_negative_run06 = negative_run06_epoched[0]
print(epochs_negative_run06.shape) #(256, 394, 478)

## Part 3: MEG Processing & Modelling

In [ ]:
## Now, we don't actually need 394 channels. We only want the MEG ones.
## From the channel editor in Brainstorm, we find that channels (1-indexed) 28-301 (274 channels) are MEG channels. So, we need to subset 
## epochs_EST_run0*. The new shape should be [number of samples, number of channels, number of epochs] where number of samples and
## number of epochs stays the same, while number of channels decreases to 274

#The M in Mepochs stands for MEG

# Run 3
Mepochs_EST_run03 = epochs_EST_run03[:,27:301,:]
print("Mepochs_EST_run03.shape:", Mepochs_EST_run03.shape) #(256, 274, 603)
Mepochs_negative_run03 = epochs_negative_run03[:,27:301,:]
print("Mepochs_negative_run03.shape:", Mepochs_negative_run03.shape) #(256, 274, 931)

# Run 4
Mepochs_EST_run04 = epochs_EST_run04[:,27:301,:]
print("Mepochs_EST_run04.shape:", Mepochs_EST_run04.shape) #(256, 274, 236)
Mepochs_negative_run04 = epochs_negative_run04[:,27:301,:]
print("Mepochs_negative_run04.shape:", Mepochs_negative_run04.shape) #(256, 274, 224)

# Run 5
Mepochs_EST_run05 = epochs_EST_run05[:,27:301,:]
print("Mepochs_EST_run05.shape:", Mepochs_EST_run05.shape) #(256, 274, 242)
Mepochs_negative_run05 = epochs_negative_run05[:,27:301,:]
print("Mepochs_negative_run05.shape:", Mepochs_negative_run05.shape) #(256, 274, 156)

# Run 6
Mepochs_EST_run06 = epochs_EST_run06[:,27:301,:]
print("Mepochs_EST_run06.shape:", Mepochs_EST_run06.shape) #(256, 274, 611)
Mepochs_negative_run06 = epochs_negative_run06[:,27:301,:]
print("Mepochs_negative_run06.shape:", Mepochs_negative_run06.shape) #(256, 274, 478)

In [ ]:
#Transpose to be (number of 1-second epochs, number of samples, number of channels)

MEG_EST_run03 = Mepochs_EST_run03.transpose(2, 0, 1) 
MEG_EST_run04 = Mepochs_EST_run04.transpose(2, 0, 1)
MEG_EST_run05 = Mepochs_EST_run05.transpose(2, 0, 1)
MEG_EST_run06 = Mepochs_EST_run06.transpose(2, 0, 1)
print(MEG_EST_run03.shape) #(603, 256, 274)
print(MEG_EST_run04.shape) #(236, 256, 274)
print(MEG_EST_run05.shape) #(242, 256, 274)
print(MEG_EST_run06.shape) #(611, 256, 274)

MEG_negative_run03 = Mepochs_negative_run03.transpose(2,0,1)
MEG_negative_run04 = Mepochs_negative_run04.transpose(2,0,1)
MEG_negative_run05 = Mepochs_negative_run05.transpose(2,0,1)
MEG_negative_run06 = Mepochs_negative_run06.transpose(2,0,1)
print(MEG_negative_run03.shape) #(931, 256, 274)
print(MEG_negative_run04.shape) #(224, 256, 274)
print(MEG_negative_run05.shape) #(156, 256, 274)
print(MEG_negative_run06.shape) #(478, 256, 274)

In [ ]:
## During preprocessing in Brainstorm, we identified the following channels to be bad channels. So, we need to remove them from the data structures above.
## MRT51 284-28=256
## MLT55 157-28=129
## MRT41 277-28=249
## MLT47 152-28=124
## MLT57 159-28=131
## MLT56 158-28=130
## MRT42 278-28=250

MEG_EST_run03= np.delete(MEG_EST_run03, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_EST_run03.shape) #(603, 256, 267)
MEG_EST_run04= np.delete(MEG_EST_run04, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_EST_run04.shape) #(236, 256, 267)
MEG_EST_run05= np.delete(MEG_EST_run05, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_EST_run05.shape) #(242, 256, 267)
MEG_EST_run06= np.delete(MEG_EST_run06, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_EST_run06.shape) #(611, 256, 267)

MEG_negative_run03 = np.delete(MEG_negative_run03, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_negative_run03.shape) #(931, 256, 267)
MEG_negative_run04 = np.delete(MEG_negative_run04, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_negative_run04.shape) #(224, 256, 267)
MEG_negative_run05 = np.delete(MEG_negative_run05, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_negative_run05.shape) #(156, 256, 267)
MEG_negative_run06 = np.delete(MEG_negative_run06, [129, 256, 249, 124, 131, 130, 250], axis=2)
print(MEG_negative_run06.shape) #(478, 256, 267)


#### Visualization of Averages:



In [ ]:
# Create a function to display the MEG time series data.

def show_time_series(data, xlabel, ylabel):
  cur = data 
  plt.plot(cur)
  plt.gca().invert_yaxis() #Invert to match the default reversed y-axis view in Brainstorm.
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.show()

##### Look at the average of positive samples:






In [ ]:
# We want to average across the first axis

average_MEG_EST_run03 = np.mean(MEG_EST_run03, axis=0)
show_time_series(average_MEG_EST_run03, "Sample number", "Teslas")

In [ ]:
# We want to average across the first axis

average_MEG_EST_run04 = np.mean(MEG_EST_run04, axis=0)
show_time_series(average_MEG_EST_run04, "Sample number", "Teslas")

In [ ]:
# We want to average across the first axis

average_MEG_EST_run05 = np.mean(MEG_EST_run05, axis=0)
show_time_series(average_MEG_EST_run05, "Sample number", "Teslas")

In [ ]:
# We want to average across the first axis

average_MEG_EST_run06 = np.mean(MEG_EST_run06, axis=0)
show_time_series(average_MEG_EST_run06, "Sample number", "Teslas")

In [ ]:
average_MEG_negative_run03 = np.mean(MEG_negative_run03, axis=0)
show_time_series(average_MEG_negative_run03, "Sample number", "Teslas")

In [ ]:
average_MEG_negative_run04 = np.mean(MEG_negative_run04, axis=0)
show_time_series(average_MEG_negative_run04, "Sample number", "Teslas")

In [ ]:
average_MEG_negative_run05 = np.mean(MEG_negative_run05, axis=0)
show_time_series(average_MEG_negative_run05, "Sample number", "Teslas")

In [ ]:
average_MEG_negative_run06 = np.mean(MEG_negative_run06, axis=0)
show_time_series(average_MEG_negative_run06, "Sample number", "Teslas")

In [ ]:
## Now, scale the data:

def scale_data(data):
  counter = 0
  return_copy = copy.deepcopy(data)
  for item in data:
    one_column = np.reshape(item, (-1, 1))
    scaler = MinMaxScaler()
    scaler.fit(one_column)
    one_column = scaler.transform(one_column)
    transformed = np.reshape(one_column, (data.shape[1], data.shape[2]))
    return_copy[counter] = transformed
    counter=counter+1
  
  return return_copy

MEG_EST_run03_scaled = scale_data(MEG_EST_run03)
MEG_EST_run04_scaled = scale_data(MEG_EST_run04)
MEG_EST_run05_scaled = scale_data(MEG_EST_run05)
MEG_EST_run06_scaled = scale_data(MEG_EST_run06)

MEG_negative_run03_scaled = scale_data(MEG_negative_run03)
MEG_negative_run04_scaled = scale_data(MEG_negative_run04)
MEG_negative_run05_scaled = scale_data(MEG_negative_run05)
MEG_negative_run06_scaled = scale_data(MEG_negative_run06)


In [ ]:
# Find the average of the scaled data

average_MEG_EST_run03_scaled = np.mean(MEG_EST_run03_scaled, axis=0)
show_time_series(average_MEG_EST_run03_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_EST_run04_scaled = np.mean(MEG_EST_run04_scaled, axis=0)
show_time_series(average_MEG_EST_run04_scaled,"Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_EST_run05_scaled = np.mean(MEG_EST_run05_scaled, axis=0)
show_time_series(average_MEG_EST_run05_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_EST_run06_scaled = np.mean(MEG_EST_run06_scaled, axis=0)
show_time_series(average_MEG_EST_run06_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_negative_run03_scaled = np.mean(MEG_negative_run03_scaled, axis=0)
show_time_series(average_MEG_negative_run03_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_negative_run04_scaled = np.mean(MEG_negative_run04_scaled, axis=0)
show_time_series(average_MEG_negative_run04_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_negative_run05_scaled = np.mean(MEG_negative_run05_scaled, axis=0)
show_time_series(average_MEG_negative_run05_scaled, "Sample number", "Amplitude")

In [ ]:
# Find the average of the scaled data

average_MEG_negative_run06_scaled = np.mean(MEG_negative_run06_scaled, axis=0)
show_time_series(average_MEG_negative_run06_scaled, "Sample number", "Amplitude")

Again, we can see that there is a difference between the IED and the non-IED samples! Now, we can move on to training the CNN with our data.

## Part 4: Hyperparameter Grid Search with Cross Validation

In [ ]:
#Create training and validation data

MEG_pos = np.concatenate((MEG_EST_run03_scaled, MEG_EST_run04_scaled, MEG_EST_run05_scaled, MEG_EST_run06_scaled))
MEG_neg = np.concatenate((MEG_negative_run03_scaled, MEG_negative_run04_scaled, MEG_negative_run05_scaled, MEG_negative_run06_scaled))
MEG_all_X = np.concatenate((MEG_pos, MEG_neg))
print(MEG_all_X.shape) #(3481, 256, 267)

MEG_y_1s = np.ones(MEG_pos.shape[0]).astype(int)
MEG_y_0s = np.zeros(MEG_neg.shape[0]).astype(int)
MEG_all_y = np.concatenate((MEG_y_1s, MEG_y_0s))
print(MEG_all_y.shape) #(3481, )

In [ ]:
# Use the train_test_split function to easily shuffle the data and obtain the training set.

MEG_X_grid_train,MEG_X_grid_val,MEG_y_grid_train,MEG_y_grid_val = train_test_split(MEG_all_X,MEG_all_y, test_size=0.01)
print(MEG_X_grid_train.shape)
print(MEG_y_grid_train.shape)

In [ ]:
# Print a few y's to make sure that the data is indeed shuffled:

print(MEG_y_grid_train[0:20])

In [ ]:
try:
  from scikeras.wrappers import KerasClassifier
except:
  !pip install scikeras
  from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def get_clf(dropout, filters, fullyneurons):
  # create model
  model = tf.keras.Sequential() #Create a sequential model

  #Convolutional layer #1
  #Padding set to same to preserve the spatial dimensions of the volume such that the output volume size matches the input volume size
  model.add(layers.Conv2D(filters, (3, 3), padding="same", input_shape=(256, 267, 1))) 
  model.add(layers.BatchNormalization())
  model.add(layers.Activation("relu"))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Dropout(dropout)) 

  #Convolutional layer #2
  model.add(layers.Conv2D(filters*2, (3, 3), padding="same")) 
  model.add(layers.BatchNormalization())
  model.add(layers.Activation("relu"))
  model.add(layers.MaxPool2D(pool_size=(2,2))) 
  model.add(layers.Dropout(dropout))

  #Convolutional layer #3
  model.add(layers.Conv2D(filters*3, (3, 3), padding="same")) 
  model.add(layers.BatchNormalization())
  model.add(layers.Activation("relu"))
  model.add(layers.MaxPool2D(pool_size=(2,2))) 
  model.add(layers.Dropout(dropout)) 

  #Flatten the volume to pass through fully connected layer
  model.add(layers.Flatten())

  #Fully connected layer #1
  model.add(layers.Dense(fullyneurons))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation("relu"))
  model.add(layers.Dropout(dropout))

  #Classification result
  model.add(layers.Dense(1, activation="sigmoid")) #2 possible output: epileptiform, or non-epileptiform. 
  return model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.1, patience=2, min_lr=0.00001, model="auto") #learning rate scheduling, decreases learning rate by 0.1 if doesn't get better for 2 epochs

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10) #stop training once val loss stops decreasing for 10 passes

cb = [reduce_lr, early_stop]

clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
    model__dropout=0.2,
    model__filters = 8, 
    model__fullyneurons = 50,
    fit__validation_split=0.2,
    verbose=False,
    epochs=100, #maximum 100 passes
    callbacks=cb,
    metrics=["accuracy", tf.keras.metrics.AUC(curve="ROC", from_logits=True), tf.keras.metrics.AUC(curve="PR", from_logits=True), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [ ]:
import sklearn.metrics

scoring_metrics = ['accuracy','roc_auc']

params = {
    'model__dropout': [0.2, 0.4],
    'model__filters': [8, 16, 32],
    'model__fullyneurons': [50, 100, 500],
}

gs = GridSearchCV(clf, params, scoring=scoring_metrics, refit='accuracy', n_jobs=1, verbose=3, cv=3)

gs.fit(MEG_X_grid_train, MEG_y_grid_train)

print(gs.best_score_, gs.best_params_)

In [ ]:
import pandas as pd

gs_dict = gs.cv_results_
del gs_dict['mean_fit_time']
del gs_dict['std_fit_time']
del gs_dict['mean_score_time']
del gs_dict['std_score_time']
del gs_dict['params']
gs_df = pd.DataFrame.from_dict(gs_dict)
gs_df.to_csv("/content/drive/MyDrive/Final_IED_Data/meg_gs_df_edited.csv") 

## Part 5: Training and Testing Model Using Optimal Hyperparameters

In [ ]:
# Create Training, Validation, and Test Sets from all of our available data
# Use 70/15/15 split since dataset is relatively small

MEG_X_train,MEG_X_val_test,MEG_y_train,MEG_y_val_test = train_test_split(MEG_all_X,MEG_all_y, test_size=0.3)
MEG_X_val,MEG_X_test,MEG_y_val,MEG_y_test = train_test_split(MEG_X_val_test,MEG_y_val_test, test_size=0.5)

print(MEG_X_train.shape)
print(MEG_y_train.shape)
print(MEG_X_val.shape)
print(MEG_y_val.shape)
print(MEG_X_test.shape)
print(MEG_y_test.shape)

# Print a few y's to make sure that the data is indeed shuffled:

print(MEG_y_test[0:10])
print(MEG_y_val[0:10])
print(MEG_y_train[0:10])

In [ ]:
pickle.dump(MEG_X_train, open("/content/drive/MyDrive/Final_IED_Data/MEG_X_train.pickle",'wb'))
pickle.dump(MEG_X_val, open("/content/drive/MyDrive/Final_IED_Data/MEG_X_val.pickle",'wb'))
pickle.dump(MEG_X_test, open("/content/drive/MyDrive/Final_IED_Data/MEG_X_test.pickle",'wb'))
pickle.dump(MEG_y_train, open("/content/drive/MyDrive/Final_IED_Data/MEG_y_train.pickle",'wb'))
pickle.dump(MEG_y_val, open("/content/drive/MyDrive/Final_IED_Data/MEG_y_val.pickle",'wb'))
pickle.dump(MEG_y_test, open("/content/drive/MyDrive/Final_IED_Data/MEG_y_test.pickle",'wb'))

In [ ]:
model = tf.keras.Sequential() #Create a sequential model

#Convolutional layer #1
#Padding set to same to preserve the spatial dimensions of the volume such that the output volume size matches the input volume size
model.add(layers.Conv2D(8, (3, 3), padding="same", input_shape=(256, 267, 1))) 
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) #Max pooling used to reduce the spatial dimensions of the output volume.
model.add(layers.Dropout(0.20)) #Prevents overfitting

#Convolutional layer #2
model.add(layers.Conv2D(16, (3, 3), padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) #Max pooling used to reduce the spatial dimensions of the output volume.
model.add(layers.Dropout(0.20)) #Prevents overfitting

#Convolutional layer #3
model.add(layers.Conv2D(32, (3, 3), padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) #Max pooling used to reduce the spatial dimensions of the output volume.
model.add(layers.Dropout(0.20)) #Prevents overfitting

#Flatten the volume to pass through fully connected layer
model.add(layers.Flatten())

#Fully connected layer #1
model.add(layers.Dense(50))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.25)) #Prevents overfitting

#Classification result
model.add(layers.Dense(1, activation="sigmoid")) #2 possible output: epileptiform, or non-epileptiform.

#We will use the Adam optimizer, and compile our model
optimizer = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy", tf.keras.metrics.AUC(curve="ROC", from_logits=True), tf.keras.metrics.AUC(curve="PR", from_logits=True), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) #Loss set to binary_crossentropy using Boston paper #Note to self: labels need to be in one-hot representation

In [ ]:
keras.backend.clear_session()

In [ ]:
epochs = 20

checkpoint = ModelCheckpoint("model_weights.h5", monitor="val_accuracy", 
                            save_weights_only=True, mode="max", verbose=1)

reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.1, patience=2, min_lr=0.00001, model="auto") #learning rate scheduling, decreases learning rate by 0.1 if doesn't get better for 2 epochs

callbacks = [checkpoint, reduce_lr]

history = model.fit(
    x = MEG_X_train,
    y = MEG_y_train,
    validation_data = (MEG_X_val, MEG_y_val),
    epochs=epochs,
    callbacks = callbacks
    )

In [ ]:
results = model.evaluate(MEG_X_test, MEG_y_test)

In [ ]:
# Save my model to my Google Drive

model.save("/content/drive/MyDrive/Final IED Detectors/Final_MEG_Model")

In [ ]:
# Use this to load the trained model
reconstructed_model = keras.models.load_model("/content/drive/MyDrive/Final IED Detectors/Final_MEG_Model")

In [ ]:
MEG_X_train = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_X_train.pickle",'rb'))
MEG_X_val = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_X_val.pickle",'rb'))
MEG_X_test = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_X_test.pickle",'rb'))
MEG_y_train = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_y_train.pickle",'rb'))
MEG_y_val = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_y_val.pickle",'rb'))
MEG_y_test = pickle.load(open("/content/drive/MyDrive/Final_IED_Data/MEG_y_test.pickle",'rb'))

In [ ]:
results_reconstructed = reconstructed_model.evaluate(MEG_X_test, MEG_y_test)

## Part 6: Visualize the Model's Classication

In [ ]:
prediction_test = model.predict(MEG_X_test)
prediction_test = (prediction_test > 0.5).astype(np.float32)
prediction_test = prediction_test.flatten()

In [ ]:
positive_indices = []
negative_indices = []

for i in range(0, len(prediction_test)):
  if prediction_test[i] == 1:
    positive_indices.append(i)
  else:
    negative_indices.append(i)

In [ ]:
predicted_test_positives = np.mean(MEG_X_test[positive_indices, :, :], axis=0)
show_time_series(predicted_test_positives, "Sample number", "Amplitude")

In [ ]:
predicted_test_negatives = np.mean(MEG_X_test[negative_indices, :, :], axis=0)
show_time_series(predicted_test_negatives, "Sample number", "Amplitude")

#### Stress testing the model on only the positive data in the test dataset




In [ ]:
ied_only_indices = []

for i in range(0, len(MEG_y_test)):
  if MEG_y_test[i] == 1:
    ied_only_indices.append(i)

In [ ]:
IED_only_prediction = model.predict(MEG_X_test[ied_only_indices,:,:])
IED_only_prediction = (IED_only_prediction > 0.5).astype(np.float32)
IED_only_prediction = IED_only_prediction.flatten()

In [ ]:
collections.Counter(IED_only_prediction)

#### Stress testing the model on only the negative data in the test dataset

In [ ]:
neg_only_indices = []

for i in range(0, len(MEG_y_test)):
  if MEG_y_test[i] == 0:
    neg_only_indices.append(i)

In [ ]:
neg_only_prediction = model.predict(MEG_X_test[neg_only_indices,:,:])
neg_only_prediction = (neg_only_prediction > 0.5).astype(np.float32)
neg_only_prediction = neg_only_prediction.flatten()

In [ ]:
collections.Counter(neg_only_prediction)